# Using _egobox_ optimizer _Egor_

## Installation

In [1]:
# %pip install egobox

We import _egobox_ as _egx_ for short.

In [2]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [3]:
# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [4]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [5]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization

In [6]:
egor = egx.Egor(xspecs_xsinx, n_cstr=n_cstr_xsinx)   # see help(egor) for options

In [7]:
egor = egx.Egor(xspecs_g24, 
                n_doe=10, 
                n_cstr=n_cstr_g24, 
                cstr_tol=[1e-3, 1e-3],
                infill_strategy=egx.InfillStrategy.WB2,
                target=-5.5,
                trego=False,
            )   

In [8]:
res = egor.minimize(g24, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 30, n_start: 20, n_doe: 10, n_cstr: 2, cstr_tol: Some([0.001, 0.001], shape=[2], strides=[1], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, q_optmod: 1, q_points: 1, infill_criterion: WB2, infill_optimizer: Cobyla, regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: Fixed { nb: 1 }, target: -5.5, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Cont(0.0, 3.0), Cont(0.0, 4.0)], seed: None, trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 }, coego: CoegoConfig { activated: false, n_coop: 5 }, cstr_infill: false, cstr_strategy: MeanConstraint }
INFO:egobox_ego.solver.egor_solver:Compute initial LHS with 10 points
INFO:egobox_ego.solver.solver_impl:Train surrogates with 10 points...
INFO:egobox_ego.solver.solver_impl:Objective clustering and training...
INFO:egobox

Optimization f=[-5.50856300e+00  2.59072729e-04  6.67071866e-04] at [2.32955191 3.17901109]
Optimization history: 
Inputs = [[1.57112671 1.7371838 ]
 [2.55902665 0.7946184 ]
 [2.03626938 3.50436102]
 [1.20996574 1.36757186]
 [0.81130086 1.15058282]
 [0.2063165  0.25721196]
 [1.03371703 3.66488816]
 [0.34291957 2.35373724]
 [2.85478864 3.15428418]
 [2.26604376 2.43991473]
 [2.27635993 3.52750422]
 [2.32955191 3.17901109]]
Outputs = [[-3.30831051e+00 -1.17086758e+00 -9.26682149e-01]
 [-3.35364505e+00 -5.29840886e+00 -1.09594723e+00]
 [-5.54063040e+00  1.49345213e+00 -4.85122161e-01]
 [-2.57753760e+00 -2.45997277e+00  8.02531409e-01]
 [-1.96188368e+00 -2.70952349e+00  4.68288195e-01]
 [-4.63528457e-01 -2.01668647e+00 -1.94084741e+01]
 [-4.69860519e+00 -3.30567074e-01  3.64730687e+00]
 [-2.69665681e+00 -2.92068224e-01 -9.83914720e+00]
 [-6.00907282e+00 -1.07552673e+01  2.86411585e+00]
 [-4.70595849e+00 -2.86982017e-01 -1.01388984e+00]
 [-5.80386415e+00  7.35984001e-01  1.15170270e-01]
 [-5

### Egor as a service: ask-and-tell interface

When the user needs to be in control of the optimization loop, `Egor` can be used as a service. 

For instance with the `xsinx` objective function, we can do:

In [9]:
xlimits = egx.to_specs([[0.0, 25.0]])
egor = egx.Egor(xlimits, seed=42) 

# initial doe
x_doe = egx.lhs(xlimits, 3, seed=42)
y_doe = xsinx(x_doe)
for _ in range(10): # run for 10 iterations
    x = egor.suggest(x_doe, y_doe)  # ask for best location
    x_doe = np.concatenate((x_doe, x))
    y_doe = np.concatenate((y_doe, xsinx(x))) 
res = egor.get_result(x_doe, y_doe)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.solver.solver_impl:Train surrogates with 3 points...
INFO:egobox_ego.solver.solver_impl:Objective clustering and training...
INFO:egobox_ego.solver.solver_impl:... Objective trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.solver.solver_computations:Infill criterion WB2 scaling is updated to 13.136295052358602
INFO:egobox_ego.solver.solver_infill_optim:Optimize infill criterion...
INFO:egobox_ego.solver.solver_impl:Train surrogates with 4 points...
INFO:egobox_ego.solver.solver_impl:Objective clustering and training...
INFO:egobox_ego.solver.solver_impl:... Objective trained (1 / Mixture[Smooth(1)])
INFO:egobox_ego.solver.solver_computations:Infill criterion WB2 scaling is updated to 21.682142952330835
INFO:egobox_ego.solver.solver_infill_optim:Optimize infill criterion...
INFO:egobox_ego.solver.solver_impl:Train surrogates with 5 points...
INFO:egobox_ego.solver.solver_impl:Objective clustering and training...
INFO:egobox_ego.solver.solver_impl:... Objective trained (1

Optimization f=[-15.12510279] at [18.9344756]


## Example 2 : Mixed-integer optimization

### Test function

In [10]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType.INT, [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [11]:
egor = egx.Egor(xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(mixint_xsinx, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 30, n_start: 20, n_doe: 3, n_cstr: 0, cstr_tol: Some([], shape=[0], strides=[0], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, q_optmod: 1, q_points: 1, infill_criterion: EI, infill_optimizer: Cobyla, regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: Fixed { nb: 1 }, target: -15.12, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Int(0, 25)], seed: None, trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 }, coego: CoegoConfig { activated: false, n_coop: 5 }, cstr_infill: false, cstr_strategy: MeanConstraint }
INFO:egobox_ego.solver.egor_solver:Compute initial LHS with 3 points
INFO:egobox_ego.solver.solver_impl:Train surrogates with 3 points...
INFO:egobox_ego.solver.solver_impl:Objective clustering and training...
INFO:egobox_ego.solver.solver_impl:... Object

INFO:egobox_ego.solver.solver_computations:Infill criterion EI scaling is updated to 0.65051575298129
INFO:egobox_ego.solver.solver_infill_optim:Optimize infill criterion...
INFO:egobox_ego.solver.solver_impl:Infill criterion EI max found = 1
INFO:egobox_ego.solver.solver_impl:+1 point(s), total: 6 points
INFO:egobox_ego.solver.egor_solver:********* End iteration 3/30 in 0.018s: Best fun(x[5])=[-14.1545328776705] at x=[19.946318386900113]
INFO:egobox_ego.solver.solver_impl:Train surrogates with 6 points...
INFO:egobox_ego.solver.solver_impl:Objective model hyperparameters optim init >>> [Full { init: [8.026648136426811], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, bounds: [(1e-6, 20.0)], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1 }]
INFO:egobox_ego.solver.solver_computations:Infill criterion EI scaling is updated to 0.9737348437861595
INFO:egobox_ego.solver.solver_infill_optim:Optimize infill criterion...
INFO:egobox_ego.solver.solver_impl:Infill criterion EI 

Optimization f=[-15.12161154] at [19.]
Optimization history: 
Inputs = [[ 1.]
 [22.]
 [11.]
 [24.]
 [21.]
 [20.]
 [19.]]
Outputs = [[  1.78601478]
 [ -7.10960014]
 [  5.1356682 ]
 [  4.91604976]
 [-11.44370682]
 [-14.15453288]
 [-15.12161154]]


## Example 3 : More mixed-integer optimization

In the following example we see we can have other special integer type cases, where a component of x can take one value out of a list of ordered values (ORD type) or being like an enum value (ENUM type). Those types differ by the processing related to the continuous relaxation made behind the scene:
* For INT type, resulting float is rounded to the closest int value,
* For ORD type, resulting float is cast to closest value among the given valid ones,
* For ENUM type, one hot encoding is performed to give the resulting value.  

### Test function

In [12]:
# Objective function which takes [FLOAT, ENUM1, ENUM2, ORD] as input
# Note that ENUM values are passed as indice value eg either 0, 1 or 2 for a 3-sized enum  
def mixobj(X):
    # float
    x1 = X[:, 0]
    #  ENUM 1
    c1 = X[:, 1]
    x2 = c1 == 0
    x3 = c1 == 1
    x4 = c1 == 2
    #  ENUM 2
    c2 = X[:, 2]
    x5 = c2 == 0
    x6 = c2 == 1
    # int
    i = X[:, 3]

    y = (x2 + 2 * x3 + 3 * x4) * x5 * x1 + (x2 + 2 * x3 + 3 * x4) * x6 * 0.95 * x1 + i
    return y.reshape(-1, 1)

### Mixed-integer optimization with _Egor_

In [13]:
xtypes = [
    egx.XSpec(egx.XType.FLOAT, [-5.0, 5.0]),
    egx.XSpec(egx.XType.ENUM, tags=["blue", "red", "green"]),
    egx.XSpec(egx.XType.ENUM, xlimits=[2]),
    egx.XSpec(egx.XType.ORD, [0, 2, 3]),
]
egor = egx.Egor(xtypes, seed=42)
res = egor.minimize(mixobj, max_iters=10)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 10, n_start: 20, n_doe: 0, n_cstr: 0, cstr_tol: Some([], shape=[0], strides=[0], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, q_optmod: 1, q_points: 1, infill_criterion: WB2, infill_optimizer: Cobyla, regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: Fixed { nb: 1 }, target: -inf, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Cont(-5.0, 5.0), Enum(3), Enum(2), Ord([0.0, 2.0, 3.0])], seed: Some(42), trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 }, coego: CoegoConfig { activated: false, n_coop: 5 }, cstr_infill: false, cstr_strategy: MeanConstraint }
INFO:egobox_ego.solver.egor_solver:Compute initial LHS with 8 points
INFO:egobox_ego.solver.solver_impl:Train surrogates with 8 points...
INFO:egobox_ego.solver.solver_impl:Objective clustering and training.

INFO:egobox_ego.solver.solver_impl:Infill criterion WB2 max found = 0.9359573658597624
INFO:egobox_ego.solver.solver_impl:+1 point(s), total: 9 points
INFO:egobox_ego.solver.egor_solver:********* End iteration 1/10 in 0.098s: Best fun(x[8])=[-14.249999999999895] at x=[-4.999999999999964, 0.49830197530794496, 0.6001658215116197, 1, 0.7420305289227407, 0.7217438683432195, 0.46254097451696013]
INFO:egobox_ego.solver.solver_impl:Train surrogates with 9 points...
INFO:egobox_ego.solver.solver_impl:Objective model hyperparameters optim init >>> [Full { init: [0.2522684285724534, 0.12435500350563795, 0.007757523783233709, 0.009108653205605466, 2.585981514816631e-6, 0.0005687770853748887, 0.010364854481736517], shape=[7], strides=[1], layout=CFcf (0xf), const ndim=1, bounds: [(1e-6, 20.0)], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1 }]
INFO:egobox_ego.solver.solver_computations:Infill criterion WB2 scaling is updated to 13.733495444813174
INFO:egobox_ego.solver.solver_infill_optim

Optimization f=[-14.25] at [-5.  2.  1.  0.]
Optimization history: 
Inputs = [[ 0.69939824  0.          0.          0.        ]
 [ 4.84411847  1.          0.          0.        ]
 [-4.75038813  1.          0.          2.        ]
 [-1.81967258  2.          1.          2.        ]
 [ 2.46052467  0.          0.          2.        ]
 [-2.82859054  0.          0.          2.        ]
 [ 2.5012666   2.          1.          0.        ]
 [-0.6935668   2.          1.          3.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          3.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]
 [-5.          2.          1.          0.        ]]
Outp

Note that `x_opt` result contains indices for corresponding optional tags list hence the second component should be read as 0="red", 1="green", 2="blue", while the third component was unamed 0 correspond to first enum value and 1 to the second one.

## Usage

In [14]:
help(egor)

Help on Egor in module builtins object:

class Egor(object)
 |  Egor(xspecs, n_cstr=0, cstr_tol=None, n_start=20, n_doe=0, doe=None, regr_spec=Ellipsis, corr_spec=Ellipsis, infill_strategy=Ellipsis, cstr_infill=False, cstr_strategy=Ellipsis, q_points=1, q_infill_strategy=Ellipsis, infill_optimizer=Ellipsis, kpls_dim=None, trego=False, coego_n_coop=0, n_clusters=1, q_optmod=1, target=Ellipsis, outdir=None, warm_start=False, hot_start=None, seed=None)
 |  
 |  Optimizer constructor
 |  n_cstr (int):
 |      the number of constraints which will be approximated by surrogates (see `fun` argument)
 |  
 |  cstr_tol (list(n_cstr + n_fcstr,)):
 |      List of tolerances for constraints to be satisfied (cstr < tol),
 |      list size should be equal to n_cstr + n_fctrs where n_cstr is the `n_cstr` argument
 |      and `n_fcstr` the number of constraints passed as functions.
 |      When None, tolerances default to DEFAULT_CSTR_TOL=1e-4.
 |  
 |  xspecs (list(XSpec)) where XSpec(xtype=FLOAT|INT|